# LangChain 101: Models - Lecture 2a
In this Colab notebook, we'll delve into the following topics:
- Understanding tokens
- Interacting directly with the model
- Exploring embeddings
- Utilizing open-source alternatives to OpenAI models
- Quantizing models for Colab execution
- Integrating LangChain seamlessly
- Initiating a large language model for CPU-only operation

Learn more about LLMs from **Models** session from [LangChain 101 course](https://github.com/IvanReznikov/DataVerse/tree/main/Courses/LangChain/Lecture2.%20Models)

### Installing dependencies

In [ ]:
!pip install -q https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl#sha256=a7a49d459bf4862f64f7bc1a68beccf8881c2fa9f3e0569608e16ba6f85ebf7b

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# install all proper packages
!pip install -q einops bitsandbytes scikit-learn seaborn matplotlib jedi
!pip install -q ctransformers Cython
!pip install -q transformers sentence_transformers safetensors
!pip install -q pydantic openai tiktoken langchain huggingface_hub

In [ ]:
if device == "cuda":
    !pip install -q auto_gptq
    !pip install -q --upgrade jax jaxlib -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [ ]:
# Dependencies versions, jic
"""
einops==0.6.1 bitsandbytes==0.41.1 scikit-learn==1.2.2 seaborn==0.12.2 matplotlib==3.7.1 jedi==0.19.0
ctransformers==0.2.23 Cython==0.29.36
transformers==4.32.0 sentence-transformers==2.2.2 safetensors==0.3.2 xformers==0.0.21
pydantic==2.2.0 openai==0.27.9 tiktoken==0.4.0 langchain==0.0.271 huggingface-hub==0.16.4
auto-gptq==0.3.0 jax==0.4.14
"""

Download huggingface embeddings now, due to conflict with preferredencoding as UTF-8

In [ ]:
import langchain

langchain.embeddings.HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
import locale

locale.getpreferredencoding = lambda: "UTF-8"

Configuring openai and huggingface keys (*optional*)

In [ ]:
import os
import openai

# You may insert your key here:
os.environ["OPENAI_API_KEY"] = "sk-..."

# You may insert your key here:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf-..."

In [ ]:
from google.colab import drive

# Check if the OpenAI API key needs to be set
if os.environ["OPENAI_API_KEY"] == "sk-...":
    # Mount Google Drive to access files
    drive.mount("/content/drive")

    # Define the file path to the OpenAI API key file
    filepath = "drive/MyDrive/Public/Colabs/Courses/LangChain/openai_course_key.key"

    # Read the API key from the file
    with open(filepath) as my_file:
        key = my_file.read()

    # Set the OpenAI API key
    openai.api_key = key
    os.environ["OPENAI_API_KEY"] = key

# Check if the Hugging Face Hub API token needs to be set
if os.environ["HUGGINGFACEHUB_API_TOKEN"] == "hf-...":
    # Mount Google Drive to access files
    drive.mount("/content/drive")

    # Define the file path to the Hugging Face Hub API token file
    filepath = "drive/MyDrive/Public/Colabs/Courses/LangChain/hf_course_key.key"

    # Read the API token from the file
    with open(filepath) as my_file:
        key = my_file.read()

    # Set the Hugging Face Hub API token
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = key

In [ ]:
import pprint

Global config for demo purposes

In [ ]:
RUN_OPENAI = False

## Tokens

Tokens are fundamental units of text that a computer program, particularly a language model, uses to process and understand language. In the context of natural language processing and machine learning, a token is typically a single word, punctuation mark, or subword unit that carries meaning.

In [ ]:
import tiktoken


def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """
    Count the number of tokens in a given text string.

    Parameters:
        string (str): The input text string.
        encoding_name (str): The name of the encoding model to be used.

    Returns:
        int: The number of tokens in the input string.
    """
    encoding = tiktoken.encoding_for_model(encoding_name)
    encoded_string = encoding.encode(string)
    return len(encoded_string)


def calculate_final_tokens(data, encoding_name="davinci"):
    """
    Calculate the total number of tokens for a list of prompts and completions.

    Parameters:
        data (list of dict): A list of dictionaries containing prompts and completions.
        encoding_name (str, optional): The name of the encoding model to be used.
            Default is "davinci".

    Returns:
        list of int: A list containing the combined token count for each prompt-completion pair.
    """
    prompt_tokens = [
        num_tokens_from_string(item["prompt"], encoding_name) for item in data
    ]
    completion_tokens = [
        num_tokens_from_string(item["completion"], encoding_name) for item in data
    ]
    res_list = [prompt_tokens[i] + completion_tokens[i] for i in range(len(data))]
    return res_list


# Sample data consisting of prompts and completions
data = [
    {
        "prompt": "Hello, how are you?",
        "completion": "I'm doing great, thanks!",
    },
    {
        "prompt": "What's your favorite color?",
        "completion": "I love blue and green!",
    },
]

# Calculate the total number of final tokens using the provided data
total_tokens = calculate_final_tokens(data)

# Print the total number of final tokens
pprint.pprint("Number of final tokens:", total_tokens)

## Speak to the model

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    """
    Generates a completion for a given prompt using the specified model.

    Args:
        prompt (str): The input prompt for generating the completion.
        model (str): The name of the model to use for generating the completion.

    Returns:
        str: The generated completion text.
    """
    # Create a list of message dictionaries with the user's prompt
    messages = [{"role": "user", "content": prompt}]

    # Generate a chat completion using OpenAI API
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # Set temperature to 0 for deterministic output
    )

    # Extract and return the generated completion content
    return response.choices[0].message["content"]

In large language models, temperature is a parameter that controls the randomness of the output. A `temperature=0` means that the model will always select the most probable next word, regardless of any other factors. This results in very deterministic output, which is often more accurate but less creative.

Learn more about temperature from **Text Generation** session from [LangChain 101 course](https://github.com/IvanReznikov/DataVerse/tree/main/Courses/LangChain/Lecture2.%20Models)

In [ ]:
if RUN_OPENAI:
    pprint.pprint(get_completion("Who was the director of Titanic?"))
    pprint.pprint(get_completion("What do they call a Big Mac in France?"))
    pprint.pprint(
        get_completion("What is the beginning of eternity, the end of time and space?")
    )
    pprint.pprint(
        get_completion(
            "What is the average age of retirement of the last 5 US presidents?"
        )
    )

As one can notice, the complexity of requests keeps growing. 

*Can you explain why?*

### Intro to prompts

In [ ]:
def generate_prompt(text, style):
    """
    Generate a prompt for converting text into a specified style.

    Parameters:
        text (str): The input text to be converted.
        style (str): The desired style for the conversion.

    Returns:
        str: The generated prompt containing the input text and style.
    """
    # Create a prompt using triple backticks and the provided text and style
    prompt = f"""Convert the text in triple backticks given to {style} style.
    text: ```{text}```
    """
    return prompt


# Example usage
generate_prompt("Hi. What's up? Let's go have lunch?", "Official Letter")

In [ ]:
if RUN_OPENAI:
    pprint.pprint(
        get_completion(
            generate_prompt("Hi. What's up? Let's go have lunch?", "Official Letter")
        )
    )

In [ ]:
if RUN_OPENAI:
    pprint.pprint(
        get_completion(
            generate_prompt(
                "Hi. What's up? Let's go have lunch?",
                "Text message with future mother-in-law",
            )
        )
    )

## Embeddings

Embeddings are compact numerical representations of words or entities that help computers understand and process language more effectively. These representations encode the meaning and context of words, allowing machines to work with language in a more meaningful and efficient way.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [ ]:
sentences = [
    "Best travel neck pillow for long flights",
    "Lightweight backpack for hiking and travel",
    "Waterproof duffel bag for outdoor adventures",
    "Stainless steel cookware set for induction cooktops",
    "High-quality chef's knife set",
    "High-performance stand mixer for baking",
    "New releases in fiction literature",
    "Inspirational biographies and memoirs",
    "Top self-help books for personal growth",
]

In [ ]:
# Creating a list of embeddings
text_embedding_list = [hf_embeddings.embed_query(s) for s in sentences]

This is how our embedding looks like:

In [ ]:
text_embedding_list[0][:10]

In [ ]:
len(text_embedding_list[0]), len(text_embedding_list)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(
    [text_embedding_list[0]], [text_embedding_list[1]]
), cosine_similarity([text_embedding_list[0]], [text_embedding_list[-1]])

As one can see, the `cosine_similarity` metric for `sentences[0]` and `sentences[1]` (0.445) is much higher than between `sentences[0]` and `sentences[9]` (0.08), symbolyzing much higher similarity.

Indeed, sentence *Best travel neck pillow for long flights* has more in common with *Lightweight backpack for hiking and travel* than *Top self-help books for personal growth*

In [ ]:
# Calculate the number of sentences
num_sentences = len(sentences)

# Initialize a similarity matrix with zeros
similarity_matrix = np.zeros((num_sentences, num_sentences))

# Calculate the cosine similarity between sentence embeddings
# Iterate through the upper triangular part of the matrix
for i in range(num_sentences):
    for j in range(i + 1, num_sentences):
        # Retrieve embeddings for the current pair of sentences
        embedding_i, embedding_j = text_embedding_list[i], text_embedding_list[j]

        # Calculate and assign the cosine similarity between the embeddings
        similarity_matrix[i, j] = cosine_similarity([embedding_i], [embedding_j])

# Copy the values from the upper triangular part to the lower triangular part
similarity_matrix += similarity_matrix.T

# Fill the diagonal of the similarity matrix with ones, indicating self-similarity
np.fill_diagonal(similarity_matrix, 1)

In [ ]:
# Create a heatmap using Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(similarity_matrix, xticklabels=False, yticklabels=False)

# Add labels and title
plt.title("Embedding Heatmap")

# Show the plot
plt.show()

In [ ]:
import pandas as pd

In [ ]:
EMBEDDING_SIZE = 75
CHUNK_SIZE = EMBEDDING_SIZE // 3

In [ ]:
sorted_var_indexes = np.argsort(np.var(text_embedding_list, axis=0))
filtered_embedding_values = np.array(np.array(text_embedding_list).T)[
    sorted_var_indexes[-EMBEDDING_SIZE:]
]

In [ ]:
_df = pd.DataFrame(filtered_embedding_values)
_df.head(3)

In [ ]:
sort_indexes = list(_df[[0, 1, 2]].mean(axis=1).sort_values().index)[:CHUNK_SIZE]
sort_indexes += [
    i for i in _df[[3, 4, 5]].mean(axis=1).sort_values().index if i not in sort_indexes
][:CHUNK_SIZE]
sort_indexes += [
    i for i in _df[[6, 7, 8]].mean(axis=1).sort_values().index if i not in sort_indexes
]

In [ ]:
sorted_text_embedding_list = filtered_embedding_values[sort_indexes]

In [ ]:
# Create a heatmap using Seaborn
plt.figure(figsize=(8, 3))
sns.heatmap(sorted_text_embedding_list.T, xticklabels=False, yticklabels=False)

# Add labels and title
plt.title("Embeddings with 75 dimensions")

# Show the plot
plt.show()

It can be seen that there is much in common within batches of 3 sentences embedding-wise

## Open-source models

In [ ]:
request_text = "List three gift ideas suitable for a 10-year-old niece"

In [ ]:
from langchain import HuggingFaceHub
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import (
    AutoTokenizer,
    TextGenerationPipeline,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)


# One can use the following code, acessing the model from HuggingFaceHub, but it's quite slow and unreliable
def inference_llm_from_hf(repo_id="google/flan-t5-xl"):
    """
    Initialize and return a Language Model from the Hugging Face Hub.

    Parameters:
        repo_id (str, optional): The ID of the model repository on Hugging Face Hub.
            Default is "google/flan-t5-xl".

    Returns:
        HuggingFaceHub: An instance of the Language Model from the Hugging Face Hub.
    """
    # Initialize a Language Model using the HuggingFaceHub class
    # Set specific model keyword arguments, such as temperature and max_length
    llm = HuggingFaceHub(
        repo_id=repo_id, model_kwargs={"temperature": 0, "max_length": 32}
    )
    return llm

In [ ]:
def get_token_ids(tokenizer, input_text):
    """
    Get token IDs from the tokenizer for a given input text.

    Parameters:
        tokenizer: The tokenizer object.
        input_text (str): The input text to tokenize.

    Returns:
        tensor: A tensor containing the token IDs of the input text.
    """
    # Use the tokenizer to convert the input text to token IDs
    # Use return_tensors="pt" to get PyTorch tensors
    token_ids = tokenizer(input_text, return_tensors="pt").input_ids
    return token_ids

### LLAMA

In [ ]:
RUN_LLAMA_GPU = False

if RUN_LLAMA_GPU and device == "cuda":
    # Import specific modules for GPU execution
    from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

    # Define the model name or path and the base model name
    model_name_or_path = "TheBloke/llama-2-13B-Guanaco-QLoRA-GPTQ"
    model_basename = "model"

    # Configure whether to use Triton
    use_triton = False

    # Initialize the tokenizer with the specified model name or path
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

    # Initialize the AutoGPTQForCausalLM model with appropriate parameters
    model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device_map="auto",
        use_triton=use_triton,
        quantize_config=None,
    )

    # Set the sequence length for the model
    model.seqlen = 8192

    # Define the prompt and prompt template
    prompt = f"{request_text}"
    prompt_template = f"""USER: {prompt}
  ASSISTANT:"""

    # Tokenize the prompt template and convert to CUDA tensor
    input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

    # Generate text using the model with specified settings
    output = model.generate(inputs=input_ids, temperature=0.1, max_new_tokens=256)

In [ ]:
if RUN_LLAMA_GPU and device == "cuda":
    pprint.pprint(tokenizer.decode(output[0]))

### Falcon

In [ ]:
RUN_FALCON_GPU = True

if device == "cuda" and RUN_FALCON_GPU:
    # Define the model name for the FALCON model
    model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

    # Configure BitsAndBytesConfig for 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    # Initialize the tokenizer using the model name
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Initialize the AutoModelForCausalLM model with quantization and specific settings
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
    )

In [ ]:
if device == "cuda" and RUN_FALCON_GPU:
    pprint.pprint(model_4bit.hf_device_map)

In [ ]:
if device == "cuda" and RUN_FALCON_GPU:
    inputs = tokenizer(request_text, return_tensors="pt").to(device)
    outputs = model_4bit.generate(**inputs, max_new_tokens=256, temperature=0.1)
    pprint.pprint(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
if device == "cuda" and RUN_FALCON_GPU:
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=300,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    pipeline(request_text)

## LangChain integration

In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain

In [ ]:
template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
if device == "cuda" and RUN_FALCON_GPU:
    # Initialize a Language Model pipeline using the HuggingFacePipeline class
    llm = HuggingFacePipeline(pipeline=pipeline)

    # Initialize an LLMChain instance with the provided prompt and llm
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    # Print the output of the llm_chain for the given request_text
    pprint.pprint(llm_chain(request_text))

## CPU-only model

In [ ]:
# check model versions https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/tree/main
if device == "cpu":
    !wget --output-document=llama-2-7b-chat.bin https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

The below cell might take a couple of minutes. Give it some time

In [ ]:
from langchain.llms import CTransformers

# Check if the device is CPU
if device == "cpu":
    # Initialize a Local CTransformers wrapper for the Llama-2-7B-Chat model
    llm = CTransformers(
        model="llama-2-7b-chat.bin",  # Location of the downloaded GGML model
        model_type="llama",  # Specify the model type as Llama
        config={
            "max_new_tokens": 256,
            "temperature": 0.1,
        },  # Set specific configuration
    )

    # Initialize an LLMChain instance with the provided prompt, llm, and verbose setting
    llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

    # Measure the time taken and print the output of llm_chain for the given request_text
    %time pprint.pprint(llm_chain(request_text))